<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# </title> Symbolic Tensor ([Quaternion](https://en.wikipedia.org/wiki/Quaternion)) Rotation

## </courtesy_remarks> Author: Ken Sible

## </brief_abstract> The following module demonstrates symbolic vector or tensor rotation using SymPy.

### </list_source_code> NRPy+ Source Code for this module:
1. [tensor_rotation.py](../edit/tensor_rotation.py); [\[**tutorial**\]](Tutorial-Symbolic_Tensor_Rotation.ipynb) </description_here> The tensor_rotation.py script will perform symbolic tensor rotation using the following function: rotate(tensor, axis, angle). 

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

0. [Preliminaries](#prelim): Derivation of quaternion rotation from matrix rotation (using [linear algebra](https://en.wikipedia.org/wiki/Linear_algebra) and [ring theory](https://en.wikipedia.org/wiki/Ring_theory))
1. [Step 1](#algorithm): Discussion of the tensor rotation algorithm (using the [SymPy](https://www.sympy.org) package for symbolic manipulation) </header_section>
1. [Step 2](#validation) Validation and demonstration of the tensor rotation algorithm (including [common subexpression elimination](https://en.wikipedia.org/wiki/Common_subexpression_elimination))
1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Preliminaries: Quaternion Rotation \[Back to [top](#toc)\]
$$\label{prelim}$$ 
Let $\vec{v}$ denote a vector in $\mathbb{R}^2$. We recall from linear algebra that $\vec{v}$ rotated about an angle $\theta$ from the x-axis, denoted $\vec{v}'$, has the following matrix formula

$$\vec{v}'=\begin{bmatrix}\cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}\vec{v}.$$

Let $z=a+bi\in\mathbb{C}$ for some $a,b\in\mathbb{R}$. Consider the corresponding (or [isomorphic](https://en.wikipedia.org/wiki/Isomorphism)) vector $\vec{z}=(a,b)\in\mathbb{R}^2$. We observe from the rotation formula that $\vec{v}'=a(\cos\theta,\sin\theta)+b(-\sin\theta,\cos\theta)$ after expanding the matrix product. Let $w=c+di\in\mathbb{C}$ for some $c,d\in\mathbb{R}$. We recall the definition of the complex product as $zw=(ac-bd)+i(ad+bc)$ where $i^2=-1$. Hence, $z'=(a+bi)(\cos\theta+i\sin\theta)=(a+bi)e^{i\theta}$ after comparing the rotated vector $\vec{v}'$ with the complex product as defined above. Therefore, the following compact formula arises for vector rotation (given the stated isomorphism between $\mathbb{C}$ and $\mathbb{R}^2$):

$$z'=e^{i\theta}z.$$

However, for vector rotation in three-dimensional space ($\mathbb{R}^3$), we curiously require a four-dimensional, non-commutative extension of complex numbers. The following discussion will provide an overview of these quaternions and their relation to vector rotation, but for a more detailed overview see [[quaternion.pdf]](http://graphics.stanford.edu/courses/cs348a-17-winter/Papers/quaternion.pdf).

$$\mathcal{H}=\{a+bi+cj+dk:a,b,c,d\in\mathbb{R}\text{ and }i^2=j^2=k^2=ijk=-1\}$$

Consider the special case of rotating a vector $\vec{v}\in\mathbb{R}^3$ through an angle $\theta$ about a normalized rotation axis $\vec{n}$ perpendicular to the vector $\vec{v}$. We usually decompose a quaternion into a scalar and vector component whenever performing vector rotation, such as decomposing $q=a+bi+cj+dk$ into $q=(a,\vec{w})$ where $\vec{w}=(b,c,d)$. For future convience, we define the following useful quaternions for vector rotation: $v=(0,\vec{v})$, $v'=(0,\vec{v}')$, and $n=(0,\vec{n})$. From the fundemental quaternion identity $i^2=j^2=k^2=ijk=-1$, we could derive quaternion multiplication (known as the Hamilton product). Let $q$ and $q'$ denote quaternions. If they have zero scalar component, then their product is $qq'=(-\vec{q}\cdot\vec{q}',\vec{q}\times\vec{q}')$, after some straightforward verification. Hence, $nv=(0,\vec{n}\times\vec{v})$ since $\vec{n}$ and $\vec{v}$ are orthogonal (perpendicular). We observe that the projection of the rotated vector $\vec{v}'$ onto $\vec{v}$ is $\cos\theta\,\vec{v}$ and the projection of $\vec{v}'$ onto $\vec{n}\times\vec{v}$ is $\sin\theta\,(\vec{n}\times\vec{v})$, and hence $\vec{v}'=\cos\theta\,\vec{v}+\sin\theta\,(\vec{n}\times\vec{v})$. We define the quaternion exponential as $e^{n\theta}=\cos\theta+n\sin\theta$, analogous to the complex exponential. Finally, we arrive at the three-dimensional vector rotation formula after comparing the rotated vector $\vec{v}'$ with the Hamilton product:

$$v'=v(\cos\theta+n\sin\theta)=e^{n\theta}v$$

The tensor rotation algorithm defined by the function rotate(tensor, axis, angle) in tensor_rotation.py (see the following code cell) does vector and tensor rotation in $\mathbb{R}^3$ about an arbitrary rotation axis (not necessarily perpendicular to the vector or tensor). For the arbitrary rotation previously described, we define the rotation quaternion as $q=e^{n(\theta/2)}$ and the conjugate of $q$ as $q^*=e^{-n(\theta/2)}$. Furthermore, the arbitrary vector rotation formula is $v'=qvq^*$ and the arbitrary tensor rotation formula is $\mathbf{M}'=(q(q\mathbf{M}q^*)^Tq^*)^T$ where the quaternion-matrix product is defined as column-wise quaternion multiplication [(source)](https://people.dsv.su.se/~miko1432/rb/Rotations%20of%20Tensors%20using%20Quaternions%20v0.3.pdf).

<a id='algorithm'></a>

# Step 1: The Tensor Rotation Algorithm \[Back to [top](#toc)\]
$$\label{algorithm}$$

In [1]:
from sympy import Quaternion as quat
from sympy import Matrix
from sympy.functions import transpose

def rotate(tensor, axis, angle):
    # Quaternion-Matrix Multiplication
    def mul(*args):
        if isinstance(args[0], list):
            q, M = args[1], args[0]
            for i, col in enumerate(M):
                M[i] = col * q
        else:
            q, M = args[0], args[1]
            for i, col in enumerate(M):
                M[i] = q * col
        return M
    # Rotation Quaternion (Axis, Angle)
    q = quat.from_axis_angle(axis, angle)
    if isinstance(tensor[0], list):
        tensor = Matrix(tensor)
        if tensor.shape != (3, 3):
            raise Exception('Invalid Matrix Dimension')
        # Rotation Formula: M' = (q.(q.M.q*)^T.q*)^T
        M = [quat(0, *tensor[:, i]) for i in range(tensor.shape[1])]
        M = mul(q, mul(M, q.conjugate()))
        for i in range(tensor.shape[1]):
            tensor[:, i] = [M[i].b, M[i].c, M[i].d]
        M = [quat(0, *tensor[i, :]) for i in range(tensor.shape[0])]
        M = mul(q, mul(M, q.conjugate()))
        for i in range(tensor.shape[0]):
            tensor[i, :] = [[M[i].b, M[i].c, M[i].d]]
        return tensor.tolist()
    else:
        if len(tensor) != 3:
            raise Exception('Invalid Vector Length')
        # Rotation Formula: v' = q.v.q*
        tensor = q * quat(0, *tensor) * q.conjugate()
        return [tensor.b, tensor.c, tensor.d]
    raise Exception('Invalid Tensor Type: Matrix or Vector')

<a id='validation'></a>

# Step 2: Validation and Demonstration \[Back to [top](#toc)\]
$$\label{validation}$$

We recall that any three-dimensional rotation can be expressed as the composition of a rotation about each coordinate axis (see [rotation theorem](https://en.wikipedia.org/wiki/Euler%27s_rotation_theorem)). Therefore, we validate our rotation algorithm using only rotations about each Cartesian axis rather than about an arbitrary axis in three-dimensional space. Consider the following vector $\vec{v}$ and matrix $\mathbf{M}$ defined below using SymPy.

In [2]:
from sympy.matrices import rot_axis1, rot_axis2, rot_axis3
from sympy import pi

v, angle = [1, 0, 1], pi/2
M = [[1, 2, 1], [0, 1, 0], [2, 1, 2]]
display(Matrix(v), Matrix(M))

Matrix([
[1],
[0],
[1]])

Matrix([
[1, 2, 1],
[0, 1, 0],
[2, 1, 2]])

We further recall that for any rotation matrix $\mathbf{R}$ and vector $\vec{v}$, the rotated vector $\vec{v}'$ has the formula $\vec{v}'=\mathbf{R}\vec{v}$ and the rotated matrix $\mathbf{M}'$ has the formula $\mathbf{M}'=\mathbf{R}\mathbf{M}\mathbf{R}^T$, where $\mathbf{R}^T$ is the transpose of $\mathbf{R}$, since rotation matrices are orthogonal, meaning that $R^{-1}=R^T$ (inverse and transpose are equivalent).

In [3]:
# vector rotation about x-axis
expected = rot_axis1(-angle) * Matrix(v)
recieved = Matrix(rotate(v, [1, 0, 0], angle))
assert expected == recieved; display(recieved)

# matrix rotation about x-axis
expected = rot_axis1(-angle) * Matrix(M) * transpose(rot_axis1(-angle))
recieved = Matrix(rotate(M, [1, 0, 0], angle))
assert expected == recieved; display(recieved)

Matrix([
[ 1],
[-1],
[ 0]])

Matrix([
[ 1, -1,  2],
[-2,  2, -1],
[ 0,  0,  1]])

In [4]:
# vector rotation about y-axis
expected = rot_axis2(-angle) * Matrix(v)
recieved = Matrix(rotate(v, [0, 1, 0], angle))
assert expected == recieved; display(recieved)

# matrix rotation about y-axis
expected = rot_axis2(-angle) * Matrix(M) * transpose(rot_axis2(-angle))
recieved = Matrix(rotate(M, [0, 1, 0], angle))
assert expected == recieved; display(recieved)

Matrix([
[ 1],
[ 0],
[-1]])

Matrix([
[ 2,  1, -2],
[ 0,  1,  0],
[-1, -2,  1]])

In [5]:
# vector rotation about z-axis
expected = rot_axis3(-angle) * Matrix(v)
recieved = Matrix(rotate(v, [0, 0, 1], angle))
assert expected == recieved; display(recieved)

# matrix rotation about z-axis
expected = rot_axis3(-angle) * Matrix(M) * transpose(rot_axis3(-angle))
recieved = Matrix(rotate(M, [0, 0, 1], angle))
assert expected == recieved; display(recieved)

Matrix([
[0],
[1],
[1]])

Matrix([
[ 1, 0, 0],
[-2, 1, 1],
[-1, 2, 2]])

The rotation algorithm does support symbolic rotation using SymPy, as demonstrated below on the 4-vector $v^\mu$ and the second rank, symmetric tensor $h^{\mu\nu}$. We remark that applying a rotation to the tensor $h^{\mu\nu}$ and following that rotation by the inverse rotation does preserve the index symmetry, as demonstrated.

In [6]:
import indexedexp as ixp
from sympy import simplify

vU  = ixp.declarerank1("vU")
hUU = ixp.declarerank2("hUU", "sym01")

expected = rot_axis1(-angle) * Matrix(vU)
recieved = simplify(Matrix(rotate(vU, [1, 0, 0], angle)))
assert expected == recieved; display(recieved)

expected = rot_axis1(-angle) * Matrix(hUU) * transpose(rot_axis1(-angle))
recieved = simplify(Matrix(rotate(hUU, [1, 0, 0], angle)))
assert expected == recieved; display(recieved)

Matrix([
[ vU0],
[-vU2],
[ vU1]])

Matrix([
[ hUU00, -hUU02,  hUU01],
[-hUU02,  hUU22, -hUU12],
[ hUU01, -hUU12,  hUU11]])

In [7]:
rot_hUU = rotate(hUU, [1, 0, 0], angle)
inv_hUU = rotate(rot_hUU, [1, 0, 0], -angle)
expected = Matrix(hUU)
# warning: may take some time to simplify matrix
recieved = simplify(Matrix(inv_hUU))
assert expected == recieved; display(recieved)

Matrix([
[hUU00, hUU01, hUU02],
[hUU01, hUU11, hUU12],
[hUU02, hUU12, hUU22]])

If the rotation algorithm is given a symbol for the rotation angle, then the resulting expression will support common subexpression elimination.

In [8]:
from sympy.abc import x
rot_hUU = Matrix(rotate(hUU, [1, 0, 0], x))
from sympy import cse
display(cse(rot_hUU)[1][0])

Matrix([
[    x2*(x5 + x6) - x4*(-x5 - x6),  x1*x10 - x3*x9,  x1*x9 + x10*x3],
[x2*(-x12 + x14) - x4*(x12 - x14), x1*x24 - x23*x3, x1*x23 + x24*x3],
[x2*(x25 + x26) - x4*(-x25 - x26), x1*x30 - x29*x3, x1*x29 + x3*x30]])

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Symbolic_Tensor_Rotation.pdf](Tutorial-Symbolic_Tensor_Rotation.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [9]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-Symbolic_Tensor_Rotation.ipynb
!pdflatex -interaction=batchmode Tutorial-Symbolic_Tensor_Rotation.tex
!pdflatex -interaction=batchmode Tutorial-Symbolic_Tensor_Rotation.tex
!pdflatex -interaction=batchmode Tutorial-Symbolic_Tensor_Rotation.tex
!rm -f Tut*.out Tut*.aux Tut*.log

This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
